# Trade accounting

This notebook decomposes gross exports into value added categories according to the Borin and Mancini (2019) trade accounting framework.

| Category | Formula | Description |
| ---- | ------- | ----------- |
| `DAVAX1` | $\mathbf{v}_s \mathbf{B}^{(s)}_{ss} \mathbf{y}_{sr}$ | Domestic value added (DVA) completed in $s$ and sent to $r$ |
| `DAVAX2` | $\mathbf{v}_s \mathbf{B}^{(s)}_{ss} \mathbf{A}_{sr} \mathbf{B}^d_{rr} \mathbf{y}_{rr}$ | DVA in intermediates sent to, completed by, and absorbed in $r$ |
| `REX1` | $\mathbf{v}_s \mathbf{B}^{(s)}_{ss} \mathbf{A}_{sr} \mathbf{B}^d_{rr} \sum_{u \neq r,s} \mathbf{y}_{ru}$ | DVA in intermediates sent to and completed by $r$ then exported to third economy |
| `REX2` | $\mathbf{v}_s \mathbf{B}^{(s)}_{ss} \mathbf{A}_{sr} \mathbf{B}^d_{rr} \sum_{u \neq r,s} \mathbf{A}_{ru} \sum_k \sum_{\ell \neq s,r} \mathbf{B}_{uk} \mathbf{y}_{k\ell}$ | DVA in intermediates sent to and re-exported by $r$ and eventually absorbed in third economy |
| `REX3` | $\mathbf{v}_s \mathbf{B}^{(s)}_{ss} \mathbf{A}_{sr} \mathbf{B}^d_{rr} \sum_{u \neq r,s} \mathbf{A}_{ru} \sum_k \mathbf{B}_{uk} \mathbf{y}_{kr}$ | DVA in intermediates sent to and re-exported by $r$ and eventually absorbed in $r$ |
| `REF1` | $\mathbf{v}_s \mathbf{B}^{(s)}_{ss} \mathbf{A}_{sr} \mathbf{B}^d_{rr} \mathbf{y}_{rs}$ | DVA in intermediates sent to and completed by $r$ then exported to $s$ |
| `REF2` | $\mathbf{v}_s \mathbf{B}^{(s)}_{ss} \mathbf{A}_{sr} \mathbf{B}^d_{rr} \sum_{u \neq r} \mathbf{A}_{ru} \sum_k \mathbf{B}_{uk} \mathbf{y}_{ks}$ | DVA in intermediates sent to and re-exported by $r$ and eventually absorbed in $s$ |
| `FVA` | $\sum_{t \neq s} \mathbf{v}_t \mathbf{B}^{(s)}_{ts} \mathbf{e}_{sr}$ | Foreign value added (FVA) in gross exports |
| `PDC1` | $\mathbf{v}_s \mathbf{B}^{(s)}_{ss} \sum_{s \neq u} \mathbf{A}_{su} \mathbf{B}_{us} \mathbf{e}_{sr}$ | Pure double counting (PDC) of domestic origin |
| `PDC2` | $\sum_{t \neq s} \mathbf{v}_t \mathbf{B}^{(s)}_{ts} \sum_{s \neq u} \mathbf{A}_{su} \mathbf{B}_{us} \mathbf{e}_{sr}$ | PDC of foreign origin |

Sectors are broken down either by exporting sectors (es) or by the origin of value added (os). As such, only the export sector breakdown is technically an exports decomposition in that the decomposed terms sum to gross exports. Both sector breakdowns aggregate to the same country-level values.

Before running this notebook, select the MRIO version to load from `data/mrio/` as well as the corresponding output filename to be saved in `data/`.

## Setup

In [1]:
import numpy as np
import pandas as pd
import duckdb
from functions import subset, asvector, zeroout, diagvec, diagmat, diagrow

### Select MRIO version

In [2]:
# input, output = 'adb-mrio.parquet', 'ta.parquet'
# input, output = 'adb-mrio62.parquet', 'ta62.parquet'
input, output = 'adb-mrio62-const.parquet', 'ta62-const.parquet'

### Parameters

In [3]:
sectors = pd.read_excel('../dicts/sectors.xlsx').drop_duplicates(subset='ind', ignore_index=True)
years = duckdb.sql(f"SELECT DISTINCT t FROM read_parquet('../data/mrio/{input}') ORDER BY t").df()['t']
rows = duckdb.sql(f"SELECT COUNT(*) FROM read_parquet('../data/mrio/{input}')").df()

N = 35                                              # Number of sectors
G = int((rows.iloc[0, 0] / len(years) - 7) / N)     # Number of countries + 1
f = 5                                               # Number of final demand components

np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

## Decompositions

In [4]:
ta_es, ta_os = pd.DataFrame(), pd.DataFrame()

for year in years:
    
    mrio = duckdb.sql(f"SELECT * EXCLUDE(t, si) FROM read_parquet('../data/mrio/{input}') WHERE t={year}").df()
    mrio = mrio.values

    x = mrio[-1][:(G*N)]
    Z = mrio[:(G*N)][:, :(G*N)]
    va = np.sum(mrio[-7:-1][:, :(G*N)], axis=0)
    Y_big = mrio[:(G*N)][:, (G*N):-1]
    Y = Y_big @ np.kron(np.eye(G), np.ones((f, 1)))
    Yd, Yf = zeroout(Y, inverse=True), zeroout(Y)
    v = np.where(x != 0, va/x, 0)
    A = Z @ np.diag(np.where(x != 0, 1/x, 0))
    Ad, Af = zeroout(A, inverse=True), zeroout(A)
    B = np.linalg.inv(np.eye(G*N) - A)
    Bd = np.linalg.inv(np.eye(G*N) - Ad)
    E = zeroout(Z @ np.kron(np.eye(G), np.ones((N, 1))) + Y)

    for s in range(1, G+1):

        # Breakdown by export sectors

        Exports = subset(E, s, -s)
        Bnots = np.linalg.inv(np.eye(G*N) - zeroout(A, s, -s))
        VB_DC = np.diag(subset(v, s) @ subset(Bnots, s, s))
        VB_FC = np.diag(subset(v, -s) @ subset(Bnots, -s, s))
        DAVAX1 = VB_DC @ subset(Y, s, -s)
        DAVAX2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ subset(Yd, -s, -s)
        REX1 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(np.sum(subset(Yf, -s, -s), axis=1))
        REX2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagmat(subset(Af, -s, 0) @ B @ subset(Y, 0, -s), offd=True)
        REX3 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagmat(subset(Af, -s, 0) @ B @ subset(Y, 0, -s))
        REF1 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(subset(Yf, -s, s))
        REF2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(subset(Af, -s, 0) @ B @ subset(Y, 0, s))
        FVA = VB_FC @ subset(E, s, -s)
        PDC1 = np.diag(subset(v, s) @ subset(Bnots, s, s) @ subset(Af, s, 0) @ subset(B, 0, s)) @ subset(E, s, -s)
        PDC2 = np.diag(subset(v, -s) @ subset(Bnots, -s, s) @ subset(Af, s, 0) @ subset(B, 0, s)) @ subset(E, s, -s)

        ta_es_s = pd.DataFrame({
            't': int(year), 's': s, 'r': np.setdiff1d(np.arange(1, G+1), s).repeat(N),
            'breakdown': 'es',
            'i': np.tile(sectors['ind'], G-1),
            'i5': np.tile(sectors['ind5'], G-1),
            'i15': np.tile(sectors['ind15'], G-1),
            'exports': asvector(Exports),
            'davax1': asvector(DAVAX1), 'davax2': asvector(DAVAX2),
            'rex1': asvector(REX1), 'rex2': asvector(REX2), 'rex3': asvector(REX3),
            'ref1': asvector(REF1), 'ref2': asvector(REF2),
            'fva': asvector(FVA), 'pdc1': asvector(PDC1), 'pdc2': asvector(PDC2)
        })
        ta_es = pd.concat([ta_es, ta_es_s], ignore_index=True)

        # Breakdown by origin sectors

        VB_DC = np.diag(subset(v, s)) @ subset(Bnots, s, s)
        VB_FC = diagrow(subset(v, -s)) @ subset(Bnots, -s, s)
        DAVAX1 = VB_DC @ subset(Y, s, -s)
        DAVAX2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ subset(Yd, -s, -s)
        REX1 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(np.sum(subset(Yf, -s, -s), axis=1))
        REX2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagmat(subset(Af, -s, 0) @ B @ subset(Y, 0, -s), offd=True)
        REX3 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagmat(subset(Af, -s, 0) @ B @ subset(Y, 0, -s))
        REF1 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(subset(Yf, -s, s))
        REF2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(subset(Af, -s, 0) @ B @ subset(Y, 0, s))
        FVA = VB_FC @ subset(E, s, -s)
        PDC1 = VB_DC @ subset(Af, s, 0) @ subset(B, 0, s) @ subset(E, s, -s)
        PDC2 = VB_FC @ subset(Af, s, 0) @ subset(B, 0, s) @ subset(E, s, -s)

        ta_os_s = pd.DataFrame({
            't': int(year), 's': s, 'r': np.setdiff1d(np.arange(1, G+1), s).repeat(N),
            'breakdown': 'os',
            'i': np.tile(sectors['ind'], G-1), 
            'i5': np.tile(sectors['ind5'], G-1), 
            'i15': np.tile(sectors['ind15'], G-1),
            'exports': np.nan,
            'davax1': asvector(DAVAX1), 'davax2': asvector(DAVAX2),
            'rex1': asvector(REX1), 'rex2': asvector(REX2), 'rex3': asvector(REX3),
            'ref1': asvector(REF1), 'ref2': asvector(REF2),
            'fva': asvector(FVA), 'pdc1': asvector(PDC1), 'pdc2': asvector(PDC2)
        })
        ta_os = pd.concat([ta_os, ta_os_s], ignore_index=True)

    print(f'{year} done')

ta = pd.concat([ta_es, ta_os], ignore_index=True)
ta.to_parquet(f'../data/{output}', index=False)

2000 done
2007 done
2008 done
2009 done
2010 done
2011 done
2012 done
2013 done
2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
2020 done
2021 done
2022 done


### View results

In [5]:
duckdb.sql(f"SELECT * FROM read_parquet('../data/{output}')").df()

,t,s,r,breakdown,i,i5,i15,exports,davax1,davax2,rex1,rex2,rex3,ref1,ref2,fva,pdc1,pdc2
0,2000,1,2,es,1,1,1,0.609776,0.163468,0.248819,0.081439,0.047470,0.000433,0.000154,0.000227,0.067301,0.000398,0.000068
1,2000,1,2,es,2,1,2,1.060181,0.022191,0.535135,0.104959,0.282094,0.003311,0.000533,0.001660,0.109518,0.000671,0.000111
2,2000,1,2,es,3,2,3,4.063309,3.371474,0.101511,0.024553,0.023295,0.000257,0.000094,0.000140,0.537463,0.003856,0.000666
3,2000,1,2,es,4,2,3,0.389450,0.157814,0.037468,0.040907,0.048079,0.000809,0.000181,0.000354,0.102946,0.000746,0.000147
4,2000,1,2,es,5,2,3,0.057060,0.021214,0.004917,0.006384,0.006618,0.000118,0.000029,0.000048,0.017569,0.000136,0.000027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4648135,2022,63,62,os,31,5,13,NaN,5.944805,16.393890,4.216543,10.588532,0.030973,2.026854,3.473147,24.908166,0.684051,0.275475
4648136,2022,63,62,os,32,5,14,NaN,2.830025,9.175777,2.492160,5.494907,0.015715,1.276437,1.841624,9.698685,0.271178,0.101371
4648137,2022,63,62,os,33,5,14,NaN,3.942084,8.267689,2.026097,4.977072,0.014512,0.979921,1.631905,4.038146,0.238738,0.044938
4648138,2022,63,62,os,34,5,15,NaN,15.129124,31.324326,8.412939,19.432405,0.055648,4.163742,6.358925,28.251645,0.901745,0.306161
